In [1]:
# 尝试自行实现基于UserCF的TopN推荐
# 数据源为movielens的1M老数据
# 由于TopN推荐只关心是否出现过评分，而非具体评分，故只需要rating的数据作训练即可

In [28]:
# 定几个global变量
K = 80 #推荐中最相似的Top用户数
N = 10 #推荐结果中最Top的item数
R = 1 # 正反馈系数
SEED = 10
TRAIN_RATE = 0.8 # 训练集比例

In [3]:
## 读取数据，数据源为movielens的1M数据
import pandas as pd
rnames = ['user_id', 'item_id', 'rating', 'timestamp']
ratings = pd.read_table('dataset/ml-1m/ratings.dat', sep='::', header=None, names=rnames, engine='python')

## 截取其中的user_id和movie_id列
df = pd.DataFrame(data = ratings, columns = ['user_id','item_id'])


## 按比例拆分train_set和test_set
## 这只是简单拆分，实际按书中描述，应当分组反复训练
import random

train_set = []
test_set = []
for index, row in df.iterrows():
    if random.random()>TRAIN_RATE:
        test_set.append([row['user_id'], row['item_id']])
    else:
        train_set.append([row['user_id'], row['item_id']])
        
train_set = pd.DataFrame(data = train_set, columns = ['user_id','item_id'])
test_set = pd.DataFrame(data = test_set, columns = ['user_id','item_id'])


In [4]:
user_list = train_set['user_id']
user_list_test = test_set['user_id']
item_list = train_set['item_id']
item_list_test = test_set['item_id']

## 建物品到用户的倒排矩阵item_user
item_user = dict()

## 同时建立user-item的关系，算模长、交并集、流行度时用
user_item = dict()

for k in range(user_list.count()):
    u = user_list.iloc[k]
    i = item_list.iloc[k]
    if i not in item_user:
        item_user[i] = set()
    item_user[i].add(u)
    
    if u not in user_item:
        user_item[u] = set()
    user_item[u].add(i)

## 测试集
item_user_test = dict()
user_item_test = dict()
for k in range(user_list_test.count()):
    u = user_list_test.iloc[k]
    i = item_list_test.iloc[k]
    if i not in item_user_test:
        item_user_test[i] = set()
    item_user_test[i].add(u)
    
    if u not in user_item_test:
        user_item_test[u] = set()
    user_item_test[u].add(i)


In [5]:
## 建立用户之间关系的稀疏矩阵

import math

C_1 = dict() # 普通的稀疏矩阵
C_2 = dict() # 对热门商品做1/log(1+N)的惩罚
for i, users in item_user.items():
    for u in users:
        C_1.setdefault(u, dict())
        C_2.setdefault(u, dict())
        for v in users:
            if u == v:
                continue
            C_1[u].setdefault(v,0)
            C_2[u].setdefault(v,0)
            C_1[u][v] += 1
            C_2[u][v] += 1 / math.log(1 + len(users))

## 计算用户相关性

W_cos = C_1.copy() # 余弦相似度
W_jaccard = C_1.copy() # Jaccard公式

for u, related_users in C_1.items():
    for v, c_uv in related_users.items():
        W_cos[u][v] = c_uv/math.sqrt(len(user_item[u])*len(user_item[v]))
        W_jaccard[u][v] = c_uv/(len(user_item[u] | user_item[v]))

W_cos_2 = C_2.copy() # 对热门有惩罚的余弦

for u, related_users in C_2.items():
    for v, c_uv in related_users.items():
        W_cos_2[u][v] = c_uv/math.sqrt(len(user_item[u])*len(user_item[v]))


In [98]:
## UserCF推荐函数
from operator import itemgetter

def recommend(u, W , topN, relatedK):
    rank = dict()
    for v, w_uv in sorted(W[u].items(), key=itemgetter(1), reverse=True)[:relatedK]:
        for i in user_item[v]:
            if i not in user_item[u]: ## 已经在user_item[u]中的物品不需要推荐
                rank.setdefault(i,0)
                rank[i] += w_uv * R
    r = sorted(rank.items(), key=itemgetter(1), reverse=True)[:topN]
    return set([i[0] for i in r])
    


In [96]:
## 计算Recall、Precision、Coverage、Popularity
## 这里定义的流行度是曾经有K个用户进行过行为并取自然对数
## evaluate作用于测试集上

method_list = ['random','most_popular','cosine','jaccard','cosine_lt']


def evaluate(method = 'cosine', topN = N, relatedK = K):
    hit = 0
    n_recall = 0
    n_precision = 0
    all_items = len(set(item_list))
    recommended_items = []
    popularity_sum = 0
    
    d = train_set.groupby(['item_id']).size()
    d = sorted(d.items(), key = itemgetter(1), reverse=True)
    
    most_popular_items = set([i[0] for i in d[:topN]])
    
    uniq_item_list = set(item_list)
    
    gini_item_list = {}
    
    for user, items in user_item_test.items():
        if method == 'cosine':
            rank = recommend(u = user, W = W_cos , topN = topN, relatedK = relatedK)
        elif method == 'jaccard':
            rank = recommend(u = user, W = W_jaccard, topN = topN, relatedK = relatedK)
        elif method == 'cosine_lt':
            rank = recommend(u = user, W = W_cos_2, topN = topN, relatedK = relatedK)
        elif method == 'most_popular':
            rank = most_popular_items
        elif method == 'random':
            rank = set(random.sample(uniq_item_list, topN))
        else: 
            rank = set()
        hit += len(rank & items)
        n_recall += len(items)
        n_precision += topN
        recommended_items += [i for i in rank]
        popularity_sum += sum([len(item_user[i]) for i in rank])
        
        for i in rank:
            gini_item_list.setdefault(i,0)
            gini_item_list[i] += 1
    
    j = 1
    n = len(gini_item_list)
    G = 0
    gini_item_list = sorted(gini_item_list.items(), key = itemgetter(1), reverse=False)
    g_sum = sum([i[1] for i in gini_item_list])
    
    for i in range(n):
        G += (2 * j - n - 1) * gini_item_list[i][1]
        j += 1
    
    return {'recall': "{0:.2%}".format(hit / (1.0 * n_recall)),
            'precision': "{0:.2%}".format(hit / (1.0 * n_precision)),
            'coverage': "{0:.2%}".format(len(set(recommended_items))/all_items),
            'gini_index': "{0:.2%}".format(G/float(n-1)/g_sum),
            'popularity': "{:.3f}".format(math.log(popularity_sum / n_precision))
           }
    

In [97]:
import prettytable
tb = prettytable.PrettyTable()
tb.field_names = ["Method", "Recall", "Precision", "Coverage", "Gini Index", 'Popularity']
for i in method_list:
    l = []
    l.append(i)
    d = evaluate(method = i)
    tb.add_row(l + list(d.values()))
print(tb)

+--------------+--------+-----------+----------+------------+------------+
|    Method    | Recall | Precision | Coverage | Gini Index | Popularity |
+--------------+--------+-----------+----------+------------+------------+
|    random    | 0.28%  |   0.91%   | 100.00%  |   13.91%   |   5.385    |
| most_popular | 2.84%  |   9.40%   |  0.27%   |   0.00%    |   7.712    |
|    cosine    | 10.14% |   33.57%  |  18.96%  |   79.09%   |   7.319    |
|   jaccard    | 10.14% |   33.57%  |  18.96%  |   79.09%   |   7.319    |
|  cosine_lt   | 10.20% |   33.77%  |  20.42%  |   78.17%   |   7.286    |
+--------------+--------+-----------+----------+------------+------------+
